# 1. Parsing

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time  # To avoid excessive requests
from tqdm import tqdm
import re

# Wikipedia URL
BASE_URL = "https://en.wikipedia.org"
LIST_URL = BASE_URL + "/wiki/List_of_highest-grossing_films"

# Fetch the page
response = requests.get(LIST_URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Locate the table (first table with class 'wikitable')
table = soup.find('table', {'class': 'wikitable'})

# Extract film data
films = []
for row in tqdm(table.find_all('tr')[1:]):  # Skip header row
    cols = row.find_all('td')
    title_elem = row.find('th')  # Title is inside <th> instead of <td>
    
    if len(cols) < 4 or not title_elem:  # Ensure we have enough columns
        continue
    
    rank = cols[0].text.strip()
    peak = cols[1].text.strip()
    title = title_elem.text.strip()
    box_office = cols[2].text.strip()
    year = cols[3].text.strip()

    # Process Box Office: remove any letters before the first '$'
    box_office = re.sub(r'^.*?(\$)', r'\1', box_office)

    # Convert year to integer if possible
    try:
        year = int(year)
    except ValueError:
        year = None
    
    # Default values
    country = "Unknown"
    director = "Unknown"

    # Visit the film page if a link is available
    link_tag = title_elem.find('a')
    film_url = BASE_URL + link_tag['href'] if link_tag else None
    if film_url:
        try:
            film_response = requests.get(film_url)
            film_soup = BeautifulSoup(film_response.text, 'html.parser')
    
            # Find the infobox (table with class 'infobox')
            infobox = film_soup.find('table', {'class': 'infobox'})
            if infobox:
                for row in infobox.find_all('tr'):
                    header = row.find('th')
                    if header:
                        header_text = header.text.strip()
                        # Extract director(s)
                        if "Directed by" in header_text:
                            td = row.find('td')
                            if td:
                                plainlist = td.find('div', class_='plainlist')
                                if plainlist:
                                    lis = plainlist.find_all('li')
                                    director = ', '.join(li.get_text(strip=True) for li in lis if li.get_text(strip=True))
                                else:
                                    director_links = td.find_all('a')
                                    if director_links:
                                        director = ', '.join(link.get_text(strip=True) for link in director_links if link.get_text(strip=True))
                                    else:
                                        director = td.get_text(separator=', ', strip=True)
                                director = re.sub(r'\[\d+\]', '', director)
                        # Extract country information
                        elif 'Countries' in header_text or 'Country' in header_text:
                            td = row.find('td')
                            if td:
                                country = td.text.strip()
                                country = re.sub(r'\[\d+\]', '', country)
                                country = country.replace('\n', ', ').strip()
        except Exception as e:
            print(f"Error fetching {title}: {e}")
        time.sleep(1)  # Prevent hitting Wikipedia too quickly

    films.append({
        "rank": rank, 
        "peak": peak, 
        "title": title, 
        "box_office_revenue": box_office,
        "release_year": year,
        "country": country,
        "director": director
    })

print(films[:5])  # Display first 5 films

# Save data to JSON file
with open("temp/films.json", "w") as f:
    json.dump(films, f, indent=4)

print("Data successfully scraped and saved to films.json!")

import sqlite3

# Database setup
conn = sqlite3.connect("films.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        rank TEXT,
        peak TEXT,
        title TEXT NOT NULL,
        box_office_revenue TEXT,
        release_year INTEGER,
        country TEXT,
        director TEXT
    )
""")

cursor.executemany("""
    INSERT INTO films (rank, peak, title, box_office_revenue, release_year, country, director)
    VALUES (:rank, :peak, :title, :box_office_revenue, :release_year, :country, :director)
""", films)

conn.commit()
conn.close()

print("Data successfully stored in films.db!")


100%|██████████| 50/50 [01:27<00:00,  1.75s/it]

[{'rank': '1', 'peak': '1', 'title': 'Avatar', 'box_office_revenue': '$2,923,706,026', 'release_year': 2009, 'country': 'United Kingdom, United States', 'director': 'James Cameron'}, {'rank': '2', 'peak': '1', 'title': 'Avengers: Endgame', 'box_office_revenue': '$2,797,501,328', 'release_year': 2019, 'country': 'United States', 'director': 'Anthony Russo, Joe Russo'}, {'rank': '3', 'peak': '3', 'title': 'Avatar: The Way of Water', 'box_office_revenue': '$2,320,250,281', 'release_year': 2022, 'country': 'United States', 'director': 'James Cameron'}, {'rank': '4', 'peak': '1', 'title': 'Titanic', 'box_office_revenue': '$2,257,844,554', 'release_year': 1997, 'country': 'United States', 'director': 'James Cameron'}, {'rank': '5', 'peak': '3', 'title': 'Star Wars: The Force Awakens', 'box_office_revenue': '$2,068,223,624', 'release_year': 2015, 'country': 'United States', 'director': 'J. J. Abrams'}]
Data successfully scraped and saved to films.json!
Data successfully stored in films.db!


In [ ]:
# load into json file from database

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time  # To avoid excessive requests
from tqdm import tqdm
import re

# Wikipedia URL
BASE_URL = "https://en.wikipedia.org"
LIST_URL = BASE_URL + "/wiki/List_of_highest-grossing_films"

# Fetch the page
response = requests.get(LIST_URL)
soup = BeautifulSoup(response.text, 'html.parser')

# Locate the table (first table with class 'wikitable')
table = soup.find('table', {'class': 'wikitable'})

# Extract film data
films = []
for row in tqdm(table.find_all('tr')[1:]):  # Skip header row
    cols = row.find_all('td')
    title_elem = row.find('th')  # Title is inside <th> instead of <td>
    
    if len(cols) < 4 or not title_elem:  # Ensure we have enough columns
        continue
    
    rank = cols[0].text.strip()
    peak = cols[1].text.strip()
    title = title_elem.text.strip()
    box_office = cols[2].text.strip()
    year = cols[3].text.strip()

    # Extract film page URL
    link_tag = title_elem.find('a')
    film_url = BASE_URL + link_tag['href'] if link_tag else None

    # Convert year to integer if possible
    try:
        year = int(year)
    except ValueError:
        year = None
    
    # Default values
    country = "Unknown"
    director = "Unknown"

    # Visit the film page if a link is available
    if film_url:
        try:
            film_response = requests.get(film_url)
            film_soup = BeautifulSoup(film_response.text, 'html.parser')
    
            # Find the infobox (table with class 'infobox')
            infobox = film_soup.find('table', {'class': 'infobox'})
            if infobox:
                for row in infobox.find_all('tr'):
                    header = row.find('th')
                    if header:
                        header_text = header.text.strip()
                        # Extract director(s)
                        # Extract director(s)
                        # Extract director(s)
                        if "Directed by" in header_text:
                            td = row.find('td')
                            if td:
                                # Check if there's a plainlist with <li> elements
                                plainlist = td.find('div', class_='plainlist')
                                if plainlist:
                                    lis = plainlist.find_all('li')
                                    director = ', '.join(li.get_text(strip=True) for li in lis if li.get_text(strip=True))
                                else:
                                    # Fallback: use <a> tags if available
                                    director_links = td.find_all('a')
                                    if director_links:
                                        director = ', '.join(link.get_text(strip=True) for link in director_links if link.get_text(strip=True))
                                    else:
                                        director = td.get_text(separator=', ', strip=True)
                                # Remove reference markers
                                director = re.sub(r'\[\d+\]', '', director)


                        # Extract country information
                        elif 'Countries' in header_text or 'Country' in header_text:
                            td = row.find('td')
                            if td:
                                country = td.text.strip()
                                country = re.sub(r'\[\d+\]', '', country)  # Remove references
                                country = country.replace('\n', ', ').strip()
        except Exception as e:
            print(f"Error fetching {title}: {e}")

        time.sleep(1)  # Prevent hitting Wikipedia too quickly

    films.append({
        "rank": rank, 
        "peak": peak, 
        "title": title, 
        "box_office": box_office, 
        "release_year": year,
        "country": country,
        "director": director
    })
    
    

100%|██████████| 50/50 [01:26<00:00,  1.74s/it]


In [2]:

print(films[:5])  # Display first 5 films

# Save data to JSON file
with open("temp/films.json", "w") as f:
    json.dump(films, f, indent=4)

print("Data successfully scraped and saved to films.json!")


[{'rank': '1', 'peak': '1', 'title': 'Avatar', 'box_office': '$2,923,706,026', 'release_year': 2009, 'country': 'United Kingdom, United States', 'director': 'James Cameron'}, {'rank': '2', 'peak': '1', 'title': 'Avengers: Endgame', 'box_office': '$2,797,501,328', 'release_year': 2019, 'country': 'United States', 'director': 'Anthony Russo, Joe Russo'}, {'rank': '3', 'peak': '3', 'title': 'Avatar: The Way of Water', 'box_office': '$2,320,250,281', 'release_year': 2022, 'country': 'United States', 'director': 'James Cameron'}, {'rank': '4', 'peak': '1', 'title': 'Titanic', 'box_office': 'T$2,257,844,554', 'release_year': 1997, 'country': 'United States', 'director': 'James Cameron'}, {'rank': '5', 'peak': '3', 'title': 'Star Wars: The Force Awakens', 'box_office': '$2,068,223,624', 'release_year': 2015, 'country': 'United States', 'director': 'J. J. Abrams'}]
Data successfully scraped and saved to films.json!


# 2. Storing

In [3]:
import sqlite3
import json

# Load data from JSON file
with open("temp/films.json", "r") as f:
    films = json.load(f)

# Database setup
conn = sqlite3.connect("films.db")
cursor = conn.cursor()

# Create table with director and country columns
cursor.execute("""
    CREATE TABLE IF NOT EXISTS films (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        rank TEXT,
        peak TEXT,
        title TEXT NOT NULL,
        box_office TEXT,
        release_year INTEGER,
        country TEXT,
        director TEXT
    )
""")

# Insert data, now including director and country
cursor.executemany("""
    INSERT INTO films (rank, peak, title, box_office, release_year, country, director)
    VALUES (:rank, :peak, :title, :box_office, :release_year, :country, :director)
""", films)

# Commit and close connection
conn.commit()
conn.close()

print("Data successfully stored in films.db!")


Data successfully stored in films.db!
